In [1]:
!pip install neo4j

In [2]:
from neo4j import GraphDatabase

In [5]:
# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j+ssc://replace with your detailsdatabases.neo4j.io"
AUTH = ("neo4j", "replace with your password)

driver = GraphDatabase.driver(URI, auth=AUTH)
driver.verify_connectivity()


In [6]:
import json

# Load the JSON file
file_path = "metro.json"
with open(file_path, "r") as file:
    data = json.load(file)

# Extract nodes
nodes = data["nodes"]
print(f"Total stations: {len(nodes)}")


Total stations: 314


In [9]:
def bulk_insert_nodes(tx, nodes):
    query = """
    UNWIND $nodes AS node
    MERGE (s:Station {id: node.id})
    SET s.name = node.text,
        s.longitude = node.longitude,
        s.latitude = node.latitude,
        s.size = node.size,
        s.x = node.x,
        s.y = node.y,
        s.visited = COALESCE(node.visited, false)
    """
    tx.run(query, nodes=nodes)

# Insert the nodes
with driver.session() as session:
    session.execute_write(bulk_insert_nodes, nodes)
    print(f"Inserted {len(nodes)} stations into Neo4j.")

Inserted 314 stations into Neo4j.


In [ ]:
with driver.session() as session:
    session.execute_write(bulk_insert_nodes, nodes)
    print(f"Inserted {len(nodes)} stations into Neo4j.")

In [10]:
def execute_query(query):
    with driver.session() as session:
        result = session.run(query)
        return [record for record in result]

# Query to count the number of Station nodes
query = "MATCH (s:Station) RETURN count(s) AS station_count"
result = execute_query(query)

# Print the result
for record in result:
    print(f"Number of Station nodes: {record['station_count']}")

Number of Station nodes: 314


In [12]:
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c  # Distance in km


In [13]:
# Generate relationships based on proximity
def generate_relationships(stations, distance_threshold=1.5):
    relationships = []
    for i, station1 in enumerate(stations):
        for j, station2 in enumerate(stations):
            if i >= j:  # Avoid duplicate pairs and self-relations
                continue
            distance = haversine(
                station1["latitude"], station1["longitude"],
                station2["latitude"], station2["longitude"]
            )
            if distance <= distance_threshold:
                relationships.append({
                    "from": station1["id"],
                    "to": station2["id"],
                    "distance": distance,
                    "line": "default"  # Replace with actual line data if available
                })
    return relationships

# Generate relationships
relationships = generate_relationships(nodes, distance_threshold=1.5)
print(f"Generated {len(relationships)} relationships.")


Generated 3004 relationships.


In [14]:
def bulk_insert_relationships(tx, relationships):
    query = """
    UNWIND $relationships AS rel
    MATCH (a:Station {id: rel.from}), (b:Station {id: rel.to})
    CREATE (a)-[:CONNECTED_TO {distance: rel.distance, line: rel.line}]->(b)
    """
    tx.run(query, relationships=relationships)

In [15]:
# Insert relationships into Neo4j
with driver.session() as session:
    session.execute_write(bulk_insert_relationships, relationships)
    print(f"Inserted {len(relationships)} relationships into Neo4j.")


Inserted 3004 relationships into Neo4j.


In [17]:
query = """
MATCH (a:Station)-[r:CONNECTED_TO]->(b:Station)
RETURN a.name AS station1, b.name AS station2, r.distance AS distance
LIMIT 10
"""
results = execute_query(query)

# Display the results
for record in results:
    print(f"Station 1: {record['station1']}, Station 2: {record['station2']}, Distance: {record['distance']} km")


Station 1: ChÃ¢tillon-Montrouge, Station 2: Malakoff - Rue Ã‰tienne Dolet, Distance: 0.6041303238388324 km
Station 1: Les Sablons, Station 2: Argentine, Distance: 1.4022020483069286 km
Station 1: Argentine, Station 2: George V, Distance: 0.8983402319230762 km
Station 1: George V, Station 2: Franklin D. Roosevelt, Distance: 0.772424239966335 km
Station 1: Franklin D. Roosevelt, Station 2: Concorde, Distance: 0.9492954048664906 km
Station 1: Concorde, Station 2: Tuileries, Distance: 0.6426824418647624 km
Station 1: Concorde, Station 2: Palais Royal - MusÃ©e du Louvre, Distance: 1.1200890687286136 km
Station 1: Tuileries, Station 2: Palais Royal - MusÃ©e du Louvre, Distance: 0.4790371660896081 km
Station 1: Tuileries, Station 2: Louvre â€” Rivoli, Distance: 0.8704348725836409 km
Station 1: Palais Royal - MusÃ©e du Louvre, Station 2: Louvre â€” Rivoli, Distance: 0.3914559993754351 km
